In [1]:
import mysql.connector as mysql
import pandas as pd

In [2]:
db_conn = mysql.connect(host = '10.4.19.215', user = 'sugengw07', password= 'sgwi2341',
database = 'mpninfo',port = '3306')

In [3]:
data_bulan = pd.read_sql('select datebayar, ket, nominal from netto2021;',con=db_conn)
data_bulan.datebayar = pd.to_datetime(data_bulan.datebayar)
data_bulan.datebayar = data_bulan.datebayar.dt.month_name()
data_bulan = data_bulan.groupby(['datebayar','ket']).sum().reset_index()
data_bulan.datebayar = data_bulan.datebayar.astype('str')
namabulan = ['January',	'February',	'March',	'April',	'May',	'June',	'July',	'August',	'September',	'October',	'November',	'December']
data_bulan.datebayar = pd.Categorical(data_bulan.datebayar,categories=namabulan,ordered=True)
data_bulan = data_bulan.sort_values(by='datebayar')
data_bulan = data_bulan.pivot_table(index=['datebayar'],columns='ket',values='nominal').reset_index()
data_bulan['Netto'] = data_bulan.MPN+data_bulan['PBK KIRIM']+data_bulan['PBK TERIMA']+data_bulan.SPM+data_bulan.SPMKP
data_bulan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   datebayar   12 non-null     category
 1   MPN         12 non-null     float64 
 2   PBK KIRIM   12 non-null     float64 
 3   PBK TERIMA  12 non-null     float64 
 4   SPM         12 non-null     float64 
 5   SPMKP       12 non-null     float64 
 6   Netto       12 non-null     float64 
dtypes: category(1), float64(6)
memory usage: 1.1 KB


In [69]:
df.datebayar = pd.to_datetime(df.datebayar)

In [70]:
df.datebayar = df.datebayar.dt.month_name()

In [73]:
df = df.groupby('datebayar').sum().reset_index()
df.datebayar = df.datebayar.astype('str')
namabulan = ['January',	'February',	'March',	'April',	'May',	'June',	'July',	'August',	'September',	'October',	'November',	'December']
df.datebayar = pd.Categorical(df.datebayar,categories=namabulan,ordered=True)
df.sort_values(by='datebayar')

,datebayar,nominal
0,January,8.637648e+11
1,February,7.963779e+11
2,March,1.075893e+12
3,April,1.386616e+12
4,May,4.998920e+11
5,June,6.958245e+11
6,July,6.393675e+11
7,August,5.260521e+11
8,September,6.303477e+11
9,October,5.200213e+11


In [72]:
df

,datebayar,nominal
0,April,1.386616e+12
1,August,5.260521e+11
2,December,3.600206e+11
3,February,7.963779e+11
4,January,8.637648e+11
5,July,6.393675e+11
6,June,6.958245e+11
7,March,1.075893e+12
8,May,4.998920e+11
9,November,8.975575e+11


In [32]:
des = df[df['BULANBAYAR']==12]

In [35]:
des.to_excel(r'E:\JUPYTER\des.xlsx',index=False)

In [18]:
map = ['411124','411124','411124','411124','411127','411127','411127','411127','411128']

In [21]:
slice = df[df['KD MAP'].isin(map)& df['bulanbayar'].isin(['8','9','10'])]
#&df['kdbayar'].isin(['101','301','311','321','419']) 

In [23]:
slice.kdbayar.value_counts()

100    4
104    4
403    3
409    3
102    1
103    1
Name: kdbayar, dtype: int64

In [32]:
df = df.groupby(['FULL','NAMA_WP']).sum().reset_index().sort_values(by='nominal',ascending=False)

In [34]:
df

,FULL,NAMA_WP,nominal,source
806,316771872003000,MITSUBISHI MOTORS KRAMA YUDHA SALES INDONESIA,1.343251e+12,1276
140,015480486007000,FASTRATA BUANA,2.242669e+11,324
233,017350976007000,ROYAL SUTAN AGUNG,2.132761e+11,1955
185,015960529011000,ASMIN BARA BRONANG,1.636182e+11,126
5,010027134007000,SOHO INDUSTRI PHARMASI,1.284115e+11,559
...,...,...,...,...
488,025456039006000,TUAH TURANGGA AGUNG,-5.825234e+09,38
682,033299553002000,MEDICA SUKSES ABADI,-6.700073e+09,101
625,031652548002000,SENTRA PRIMER SOLUSINDO,-3.656935e+10,96
567,029985207007000,PARIT PADANG GLOBAL,-4.223067e+10,58291


In [35]:
netto_top100=df.nlargest(100,columns='nominal')

In [36]:
netto_top100

,FULL,NAMA_WP,nominal,source
806,316771872003000,MITSUBISHI MOTORS KRAMA YUDHA SALES INDONESIA,1.343251e+12,1276
140,015480486007000,FASTRATA BUANA,2.242669e+11,324
233,017350976007000,ROYAL SUTAN AGUNG,2.132761e+11,1955
185,015960529011000,ASMIN BARA BRONANG,1.636182e+11,126
5,010027134007000,SOHO INDUSTRI PHARMASI,1.284115e+11,559
...,...,...,...,...
788,315620369005000,GENESA DIRGANTARA,8.702814e+09,62
290,018866780046000,LINESIA ADOFA LESTARI,8.691336e+09,187
483,025088949002000,MAJU JAYA NIAGATAMA,8.641138e+09,161
395,022458137007000,"BUYUNG POETRA SEMBADA, TBK",8.467428e+09,142


In [37]:
netto_top100.to_excel(r'D:\DATA KANTOR\SQL\netto_top100.xlsx',index=False)

In [ ]:
data.datebayar = pd.to_datetime(data.datebayar)
data = data.set_index('datebayar').sort_index()

In [ ]:
datasdokto = data.loc['2021-01-01':'2021-10-31']

In [ ]:
datasdokto.nominal.sum()

In [ ]:
bruto = data[data['ket']=='MPN']

In [ ]:
bruto.nominal.sum()

In [ ]:
ppmpkmq = '''SELECT A.NPWP,
       A.KPP,
       A.CABANG,
       A.NAMA,
       A.KDMAP,
       A.KDBAYAR,
       MASA,
       A.TAHUN,
       TAHUNBAYAR,
       BULANBAYAR,
       DATEBAYAR,
       NOMINAL,
       CASE
         WHEN SUBSTRING(A.KDMAP,1,5) IN ('41131','41111') THEN 'PPM'
         WHEN TAHUNBAYAR = A.TAHUN THEN 'PPM'
         WHEN TAHUNBAYAR - A.TAHUN IN (0,1) AND A.KDMAP IN ('411125','411126') AND A.KDBAYAR IN ('200','199','310','320','390','500','501') THEN 'PPM'
         WHEN (TAHUNBAYAR - A.TAHUN IN (0,1) AND MASA = 12) THEN 'PPM'
         WHEN A.TAHUN > TAHUNBAYAR THEN 'PPM'
         ELSE 'PKM'
       END FLAG_PPM_PKM,
       NIPPJ,
       ntpn,
       nosk,
       E.SEKTOR,
       URAIAN,
       F.NAMA AS NAMA_KLU,
       F.SEKTOR AS SEKTOR_KLU
FROM MPN A
  LEFT JOIN WP B
         ON A.NPWP = B.NPWP
        AND A.KPP = B.KPP
        AND A.CABANG = B.CABANG
  LEFT JOIN MAP E
         ON A.KDMAP = E.KDMAP
        AND A.KDBAYAR = E.KDBAYAR
  LEFT JOIN KLU F ON B.KLU = F.KODE
WHERE TAHUNBAYAR = 2021;'''

In [27]:
db_conn = mysql.connect(host = '10.4.19.215', user = 'sugengw07', password= 'sgwi2341',
database = 'mpninfo',port = '3306')
#with open(r'sql\PPM_PKM_2021.sql','r') as query:
    #penerimaan = query.read()
ppmpkm = pd.read_sql(ppmpkmq,db_conn)
mf = pd.read_excel(r'MFWP/MFWP.xlsx',usecols=[
'NPWP','NAMA_WP','NAMA_AR','SEKSI'], dtype={'NPWP':'str'},engine='openpyxl')

In [ ]:
ppmpkm.DATEBAYAR = pd.to_datetime(ppmpkm.DATEBAYAR )

In [ ]:
ppmpkm=ppmpkm.set_index('DATEBAYAR').sort_index()

In [ ]:
ppmpkm.BULANBAYAR = ppmpkm.BULANBAYAR.astype('str')

In [33]:
slice = pd.merge(ppmpkm,mf, left_on='NPWP',right_on='NPWP',how='left')

In [34]:
slice = slice.filter(['NAMA_WP','BULANBAYAR','KDMAP','NAMA','NOMINAL'])

In [35]:
slice = slice.groupby(['NAMA_WP']).sum().reset_index().sort_values(by = 'NOMINAL',ascending=False)

In [36]:
slice

,NAMA_WP,BULANBAYAR,NOMINAL
259,FASTRATA BUANA,78761,9.236137e+12
22,AKUR PRATAMA,22581,3.849663e+12
673,PARIT PADANG GLOBAL,8158650,3.691865e+12
604,MITSUBISHI MOTORS KRAMA YUDHA SALES INDONESIA,6799,1.249639e+12
571,MENSA BINASUKSES,2058903,9.402395e+11
...,...,...,...
202,DANIEL ROOSENO,20,2.000000e+05
82,AZIZAH NURAINI,18,2.000000e+05
14,AFIYAH HASANAH UTAMA,10,1.000000e+05
332,HASIL BUMI MAKMUR,6,1.000000e+05


In [25]:
slice.to_clipboard()

In [ ]:
sdoktober = ppmpkm.loc['2021-01-01':'2021-10-31']

In [ ]:
ppm = sdoktober[sdoktober['FLAG_PPM_PKM']=='PPM']
pkm = sdoktober[sdoktober['FLAG_PPM_PKM']=='PKM']

In [ ]:
print('{:,.0f}'.format(ppm.NOMINAL.sum()))

In [ ]:
print('{:,.0f}'.format(pkm.NOMINAL.sum()))

In [ ]:
spmkp = pd.read_sql('select * from spmkp;',db_conn)

In [ ]:
spmkp.tahun = spmkp.tahun.astype('str')

In [ ]:
spmkp = spmkp[(spmkp['bulan']<11) & (spmkp['tahun']=='2021')]

In [ ]:
print('{:,.0f}'.format(spmkp.nominal.sum()))

In [ ]:
ppm = ppm.filter(['NAMA_SEKSI','NAMA_AR','NPWP_FULL', 'NAMA','NTPN','NOMINAL'])
pkm = pkm.filter(['NAMA_SEKSI','NAMA_AR','NPWP_FULL', 'NAMA','NTPN','NOMINAL'])

In [ ]:
ppm= ppm.groupby(['NAMA_AR','NPWP_FULL']).sum().reset_index()
pkm= pkm.groupby(['NAMA_AR','NPWP_FULL']).sum().reset_index()

In [ ]:
ppm.to_excel('PPM_Oktober.xlsx',index=False)

In [ ]:
pkm.to_excel('PKM_Oktober.xlsx',index=False)

In [ ]:
print('{:,.0f}'.format(pkm.NOMINAL.sum()))

In [ ]:
smo = data[data['BULANBAYAR'].isin([6,7,8,9,10])]
print('{:,.0f}'.format(smo.NOMINAL.sum()))

In [ ]:
mpn.nominal.sum()

In [ ]:
data.to_excel('PPMPKM_Juni_Oktober2021.xlsx',index=False)

In [ ]:
bruto21_kueri = '''select sum(nominal) as jumlah from mpn
where datebayar between '2020-12-31' and curdate() ;'''
bruto20_kueri = '''select sum(nominal) as jumlah from mpn
where datebayar > '2019-12-31' and
datebayar <= date_sub(curdate(),interval 1 year);'''

bruto21 = pd.read_sql(bruto21_kueri,db_conn)
bruto20 = pd.read_sql(bruto20_kueri,db_conn)

In [ ]:
bruto21.subtract(bruto20)

In [ ]:
(bruto21.subtract(bruto20))/bruto20['jumlah'].sum()

In [ ]:

with open(r'sql\mpnspmpbkspmkp.sql','r') as query:
    penerimaan = query.read()
data21 = pd.read_sql(penerimaan,db_conn)
data21['NPWP_FULL'] = data.npwp+data.kpp+data.cabang
mf = mfwp()
data21 = pd.merge(data21,mf, left_on='NPWP_FULL',right_on='NPWP', how='left')
data21.drop('nama',axis=1,inplace=True)
    
with open(r'sql\mpnspmpbkspmkp.sql','r') as query:
    penerimaan = query.read()
data = pd.read_sql(penerimaan,db_conn)
data['NPWP_FULL'] = data.npwp+data.kpp+data.cabang
mf = mfwp()
data = pd.merge(data,mf, left_on='NPWP_FULL',right_on='NPWP', how='left')
data.drop('nama',axis=1,inplace=True)
    
def mfwp():
    mf = pd.read_excel(r'MFWP/MFWP+klu.xlsx',usecols=[
        'NPWP','NAMA_WP','NAMA_AR','SEKSI','NAMA_KLU'], dtype={'NPWP':'str'},engine='openpyxl')
    return mf

In [ ]:
mpn21 = netto21()
mpn20 = netto20()